# Data Split
데이터 분할은 추천 시스템을 진행하면서 가장 중요한 과제 중 하나이다. 분할 전략은 결과에 큰 영향을 미치기 때문에 신중하게 고려해야 한다.   
이 노트에선 특정한 시나리오마다 다른 분할 전략들을 적용하는 방법을 소개한다.

In [1]:
# Global settings
import sys
import pyspark
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from recommenders.utils.spark_utils import start_or_get_spark
from recommenders.datasets.download_utils import maybe_download
from recommenders.datasets.python_splitters import python_random_split, python_chrono_split, python_stratified_split
from recommenders.datasets.spark_splitters import spark_random_split

## 1. Data preparation
MovieLens-100k 데이터셋을 이용하여 진행한다.

In [2]:
### 1.1 Get data and understanding
filepath = maybe_download('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', 'ml-100k.data')
data = pd.read_csv(filepath, sep='\t', names=['UserID', 'MovieID', 'Rating', 'Timestamp'])
data.head()

100%|█████████████████████████████████████| 1.93k/1.93k [00:01<00:00, 1.39kKB/s]


,UserID,MovieID,Rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
print("Total number of ratings are\t{}".format(data.shape[0]),
    "Total number of users are\t{}".format(data['UserID'].nunique()),
    "Total number of items are\t{}".format(data['MovieID'].nunique()),
    sep="\n")

Total number of ratings are	100000
Total number of users are	943
Total number of items are	1682


In [ ]:
### 1.2 Data transformation
# 원본의 timestamp를 ISO 포맷으로 바꾼다.
data['Timestamp'] = data.apply(lambda x:datetime.strftime(datetime(1970, 1, 1, 0, 0, 0) +
                                                         timedelta(seconds=x['Timestamp'].item()),
                                                          "%Y-%m-%d %H:%M:%S"), axis=1)

In [9]:
data.head()

,UserID,MovieID,Rating,Timestamp
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


## 2. Experimentation protocol
Experimentation protocol은 일반적으로 특정한 추천 시나리오에 대한 합리적인 평가를 선호하도록 설정된다. 예를 들어,
- Recommender-A는 사람들의 collaborative 평점 유사도를 사용해 영화를 추천한다. 그 검증이 통계적임을 확실히 하기 위해, 모델 빌딩과 테스트에서 같은 유저 표본을 사용해야 하고 계층화된 분할 전략을 채용해야 한다.
- Recommender-B는 고객들에게 패션 상품을 추천한다. 이 추천에 대한 검증은 고객의 구매력의 시간 의존성을 고려해야 한다. 고객의 패션 취향이 시간이 지남에 따라 변하기 때문이다. 이 경우엔 시간순의 분할 전략을 사용해야 한다.

## 3. Data split

### 3.1 Random split
Random split은 주어진 비율로 단순히 주어진 데이터셋을 나눈다.

In [11]:
data_train, data_test = python_random_split(data, ratio=0.7)
data_train.shape[0], data_test.shape[0]

(70000, 30000)

In [13]:
# Multi-split
data_train, data_valid, data_test = python_random_split(data, ratio=[0.6, 0.2, 0.2])
data_train.shape[0], data_valid.shape[0], data_test.shape[0]

(60000, 20000, 20000)

### 3.2 Chronological split
Chronological split은 데이터셋을 timestamp 기준으로 나눈다.

**3.2.1 'Filter by'**
Chrono splitting은 'user' 기준일 수도, 'item' 기준일 수도 있다.   
기준이 'user'이고 `ratio`가 0.7인 경우는 데이터셋에서 각 유저의 앞쪽 70%의 데이터와 나머지 30%의 데이터로 나눠지는 것이다. Chronological splitting은 시간에 의존적이기 때문에 '랜덤'이 아닌 것에 신경쓸 필요 없다.

In [14]:
data_train, data_test = python_chrono_split(data, ratio=0.7, filter_by='user',
                                           col_user='UserID', col_item='MovieID', col_timestamp='Timestamp')
data_train[data_train['UserID']==1].tail(10)

,UserID,MovieID,Rating,Timestamp
1989,1,90,4,1997-11-03 07:31:40
11807,1,219,1,1997-11-03 07:32:07
50026,1,167,2,1997-11-03 07:33:03
16314,1,230,4,1997-11-03 07:33:40
51295,1,35,1,1997-11-03 07:33:40
43280,1,162,4,1997-11-03 07:33:40
202,1,61,4,1997-11-03 07:33:40
820,1,265,4,1997-11-03 07:34:01
11154,1,112,1,1997-11-03 07:34:01
45732,1,57,5,1997-11-03 07:34:19


In [16]:
print(len(data_train[data_train['UserID']==1]), len(data_test[data_test['UserID']==1]))

190 82


**3.3.2 Min-rating filter**
Min-rating filter는 chronological splitter를 사용해 분할되기 전에 데이터에 적용된다. 이 작업을 하는 이유는, multi-split에선 데이터에 있는 user/item의 평점의 수가 충분해야 하기 때문이다.   
예를 들어, 적어도 10개의 평점을 갖고 있는 사용자에게만 적용된 분할은 다음과 같다.   
분할된 데이터의 수의 합은 원본 데이터의 수와 다를 수 있다.

In [17]:
data_train, data_test = python_chrono_split(data, filter_by='user', min_rating=10, ratio=0.7,
                                           col_user='UserID', col_item='MovieID', col_timestamp='Timestamp')
data_train.shape[0] + data_test.shape[0], data.shape[0]

(100000, 100000)

### 3.4 Data split in scale
*Scalable splitting*을 위해선 *Spark DataFrame*을 사용한다. 이를 통해 Spark cluster 전체에 분산된 대규모 데이터셋에서 분할 작업을 수행할 수 있다.   
예를 들어, 주어진 Spark DataFrame에서 랜덤 분할하는 방법은 아래와 같다. 단순한 예제를 위해 Pandas DataFrame에 있는 MovieLens 데이터를 Spark DataFrame으로 바꿔 사용했다.

In [20]:
spark = start_or_get_spark()
data_spark = spark.read.csv(filepath)
ds_train, ds_test = spark_random_split(data_spark, ratio=0.7)
ds_train.count(), ds_test.count()

(69941, 30059)

Spark 랜덤 분할은 결정론적 결과를 보장하지는 않는다. 이로 인해, 데이터가 상대적으로 적을 때 정확한 불할을 원한다면 문제가 생길 수 있다.

# Data Transform (Collaborative filtering)
보통 실제 세계의 데이터셋에선 유저들이 아이템들에 대해 다른 상호작용의 타입을 가질 수 있다. 게다가 같은 타입의 상호작용이 한 번 이상 기록될 수도 있다. 실질적인 추천 시스템 디자인에서 이런 문제는 전형적임을 기억하면서, 이 노트북에서 서로 다른 시나리오들에 사용될 수 있는 *Data Transformation* 기술을 공유한다.   
특히, 이 노트북에 나오는 내용들은 collaborative filtering algorithms에만 적용 가능한다.

In [21]:
import sys
import pandas as pd
import numpy as np
import datetime
import math

## 1. Data creation
이 노트북의 설명을 위해 두 더미 데이터셋을 생성한다.
### 1.1 Explicit feedback
*Explicit feedback* 시나리오에서, 유저들과 아이템 사이의 상호작용은 숫자로 나타내지거나 / 서수적인 평점 또는 '좋음' 이나 '싫음'의 이분적인 선호도이다.   
아래에선 explicit rating type의 더미 데이터를 보여준다.
- ID가 각각 1,2,3인 3명의 유저와 ID가 각각 1,2,3인 3개의 아이템
- 아이템들은 유저들에 의해 한번만 평가된다. 유저들이 아이템과 여러 시간대에 상호작용 해도 평점은 유지된다. 이런 방식은 영화 추천같은 데서 보이는데, 유저들의 평가가 짧은 시간에 극적으로 변하지 않기 때문이다.
-

In [22]:
data1 = pd.DataFrame({'UserID':[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3],
                     'ItemID': [1, 1, 2, 2, 2, 1, 2, 1, 2, 3, 3, 3, 3, 3, 1],
                     'Rating': [4, 4, 3, 3, 3, 4, 5, 4, 5, 5, 5, 5, 5, 5, 4],
                     'Timestamp':['2000-01-01', '2000-01-01', '2000-01-02', '2000-01-02', '2000-01-02',
        '2000-01-01', '2000-01-01', '2000-01-03', '2000-01-03', '2000-01-03',
        '2000-01-01', '2000-01-03', '2000-01-03', '2000-01-03', '2000-01-04']})

### 1.2 Implicit feedback
많은 경우에 유저들에 의한 외재적인 평점이나 선호가 없이 내재적인 상호작용이 있다. 웹사이트에서 상품을 구매하거나, 어떤 아이템을 클릭하거나 등등 이런 정보는 유저의 아이템에 대한 내재적인 선호 방식을 반영한다.   
아래는 implicit feedback 시나리오의 데이터셋을 보여준다.
- ID가 각각 1,2,3인 3명의 유저와 ID가 각각 1,2,3인 3개의 아이템
- 유저와 아이템 사이의 **click, add** and **purchase** 3가지의 이벤트 타입의 상호작용을 보여준다.
- **time-spent on visiting a site before clicking**와 같은 타입의 상호작용을 고려할 수도 있다.

In [38]:
data2 = pd.DataFrame({"UserID": [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3],
                      "ItemID": [1, 1, 2, 2, 2, 1, 2, 1, 2, 3, 3, 3, 3, 3, 1],
                      "Type": ['click', 'click', 'click', 'click', 'purchase',
                               'click', 'purchase', 'add', 'purchase', 'purchase',
                               'click', 'click', 'add', 'purchase', 'click'],
                      "Timestamp": ['2000-01-01', '2000-01-01', '2000-01-02', '2000-01-02', '2000-01-02',
                                    '2000-01-01', '2000-01-01', '2000-01-03', '2000-01-03', '2000-01-03',
                                    '2000-01-01', '2000-01-03', '2000-01-03', '2000-01-03', '2000-01-04']})

## Data transformation
많은 collaborative filtering 알고리즘은 user-item 희소 행렬을 기반으로 만들어졌다. 추천 시스템을 만드는 입력 데이터는 유니크한 user-item 짝들을 포함할 필요가 있다.   
For explicit feedback datasets, this can simply be done by deduplicating the repeated user-item-rating tuples.

In [34]:
data1.duplicated().value_counts()

False    10
True      5
dtype: int64

In [36]:
data1.drop_duplicates(inplace=True)
data1

,UserID,ItemID,Rating,Timestamp
0,1,1,4,2000-01-01
2,1,2,3,2000-01-02
5,2,1,4,2000-01-01
6,2,2,5,2000-01-01
7,2,1,4,2000-01-03
8,2,2,5,2000-01-03
9,2,3,5,2000-01-03
10,3,3,5,2000-01-01
11,3,3,5,2000-01-03
14,3,1,4,2000-01-04


*Implicit feedback*을 사용하는 경우엔, 실제 비즈니스 사용자 사례의 요구 사항에 따라 deduplication을 수행하는 몇 가지의 방법이 있다. 
### 2.1 Data aggregation(종합)
일반적으로, 사용자는 데이터를 집계하여 선호도를 나타내는 몇 개의 점수를 생성한다.(*SAR*같은 알고리즘들에서 이 점수는 *affinity score*라고 불린다.)
Affinity scores는 explicit data set의 평점과 값의 분포 측면에서 다르다. 보통 *ordinal regression* 문제라 하는데, the algorithm used for training a recommender should be carefully chosen to consider the distribution of the affinity scores rather than discrete integer values.   
**2.2.1 Count**   
Affinity scores를 만드는 가장 간단한 방법은 유저와 아이템 사이 상호작용의 수를 세는 것이다.

In [40]:
data2

,UserID,ItemID,Type,Timestamp
0,1,1,click,2000-01-01
1,1,1,click,2000-01-01
2,1,2,click,2000-01-02
3,1,2,click,2000-01-02
4,1,2,purchase,2000-01-02
5,2,1,click,2000-01-01
6,2,2,purchase,2000-01-01
7,2,1,add,2000-01-03
8,2,2,purchase,2000-01-03
9,2,3,purchase,2000-01-03


In [41]:
data2_cnt = data2.groupby(['UserID','ItemID']).agg({'Timestamp':'count'}).reset_index()
data2_cnt.columns = ['UserID', 'ItemID', 'Affinity']
data2_cnt

,UserID,ItemID,Affinity
0,1,1,2
1,1,2,3
2,2,1,2
3,2,2,2
4,2,3,1
5,3,1,1
6,3,3,4


**2.2.1 Weighted count**   
서로 다른 상호작용의 유형을 카운트 집계에서 가중치로 고려하는 것은 유용하다. 세 가지 다른 타입의 상호작용에 각각 가중치를 가정하는 방법은 다음과 같다.

In [42]:
data2_w = data2.copy()

conditions = [data2_w['Type']=='click', data2_w['Type']=='add', data2_w['Type']=='purchase']
choices = [1,2,3]

data2_w['Weight'] = np.select(conditions, choices, default='black')
data2_w['Weight'] = pd.to_numeric(data2_w['Weight'])
data2_w

,UserID,ItemID,Type,Timestamp,Weight
0,1,1,click,2000-01-01,1
1,1,1,click,2000-01-01,1
2,1,2,click,2000-01-02,1
3,1,2,click,2000-01-02,1
4,1,2,purchase,2000-01-02,3
5,2,1,click,2000-01-01,1
6,2,2,purchase,2000-01-01,3
7,2,1,add,2000-01-03,2
8,2,2,purchase,2000-01-03,3
9,2,3,purchase,2000-01-03,3


In [43]:
data2_wcnt = data2_w.groupby(['UserID', 'ItemID'])['Weight'].sum().reset_index()
data2_wcnt.columns = ['UserID', 'ItemID', 'Affinity']
data2_wcnt

,UserID,ItemID,Affinity
0,1,1,2
1,1,2,5
2,2,1,3
3,2,2,6
4,2,3,3
5,3,1,1
6,3,3,7


**2.2.2 Time dependent count**   
많은 시나리오에서, 시간에 따라 변하는 유저의 관심사를 캐치하는 collaborative filtering 모델을 위한 데이터셋 준비에서 시간 의존성은 중요한 역할을 한다. 시간 의존적 count를 얻기 위한 일반적인 방법 중 하나는 시간가치 축소 요소를 추가하는 것이다. 이 방법은 *SAR*에서 사용된다.   
Formula for getting affinity score for each user-item pair is
where is the affinity score, is the interaction weight, is a reference time, is the timestamp for the -th interaction, and

is a hyperparameter that controls the speed of decay.

The following shows how SAR applies time decay in aggregating counts for the implicit feedback scenario.   

아래는 *SAR*에서 어떻게 적용하는 지를 보여준다. 반감기 파라미터를 5일로 설정하고, 시간 참조로 데이터셋의 가장 늦은 시간을 사용한다.

In [47]:
T = 5
t_ref = pd.to_datetime(data2_w['Timestamp']).max()

# Calculate the weighted count with time decay
data2_w['Timedecay'] = data2_w.apply(
lambda x:x['Weight']*np.power(0.5, (t_ref-pd.to_datetime(x['Timestamp'])).days / T), axis=1)

data2_w

,UserID,ItemID,Type,Timestamp,Weight,Timedeacy,Timedecay
0,1,1,click,2000-01-01,1,0.659754,0.659754
1,1,1,click,2000-01-01,1,0.659754,0.659754
2,1,2,click,2000-01-02,1,0.757858,0.757858
3,1,2,click,2000-01-02,1,0.757858,0.757858
4,1,2,purchase,2000-01-02,3,2.273575,2.273575
5,2,1,click,2000-01-01,1,0.659754,0.659754
6,2,2,purchase,2000-01-01,3,1.979262,1.979262
7,2,1,add,2000-01-03,2,1.741101,1.741101
8,2,2,purchase,2000-01-03,3,2.611652,2.611652
9,2,3,purchase,2000-01-03,3,2.611652,2.611652


유저-아이템 쌍의 Affinity scores는 `Timedecay` 칼럼의 값들을 합하여 계산할 수 있다.

In [48]:
data2_wt = data2_w.groupby(['UserID', 'ItemID'])['Timedecay'].sum().reset_index()
data2_wt.columns = ['UserID', 'ItemID', 'Affinity']
data2_wt

,UserID,ItemID,Affinity
0,1,1,1.319508
1,1,2,3.789291
2,2,1,2.400855
3,2,2,4.590914
4,2,3,2.611652
5,3,1,1.000000
6,3,3,5.883057


### 2.2 Negative sampling
앞의 집계는 "number of interaction times", "weights", "time decay" 등으로 user-item 상호작용이 선호도로 변환될 수 있다는 가정에 기반한다. 이런 가정은 편향될 수 있고, 상호작용 그 자체만이 중요할 수도 있다. 즉, 원래의 implicit 상호작용이 상호작용이 있었냐 없었냐를 기준으로 1,0으로 이진분류 될 수 있다는 것이다.

In [49]:
data2_b = data2[['UserID', 'ItemID']].copy()
data2_b['Feedback'] = 1
data2_b = data2_b.drop_duplicates()
data2_b

,UserID,ItemID,Feedback
0,1,1,1
2,1,2,1
5,2,1,1
6,2,2,1
9,2,3,1
10,3,3,1
14,3,1,1


'Negative sampling'은 부정적인 피드백을 샘플링하는 방법이다. 역시 시나리오마다 다르게 정의될 수 있다. 예를 들어, 이 경우엔 유저가 상호작용 하지 않은 아이템은 싫어하는 아이템이라고 판단한다. 좀 강한 가정일 수 있지만, **유저와 아이템 사이의 상호작용의 수가 그렇게 많지 않은 경우** 모델을 만들 때 합리적인 가정이다.

In [55]:
users = data2['UserID'].unique()
items = data2['ItemID'].unique()

interaction_list = []
for user in users:
    for item in items:
        interaction_list.append([user, item, 0])
data_all = pd.DataFrame(data=interaction_list, columns=['UserID', 'ItemID', 'FeedbackAll'])

In [57]:
data2_ns = pd.merge(data_all, data2_b, on=['UserID', 'ItemID'], how='outer').fillna(0).drop('FeedbackAll', axis=1)
data2_ns

,UserID,ItemID,Feedback
0,1,1,1.0
1,1,2,1.0
2,1,3,0.0
3,2,1,1.0
4,2,2,1.0
5,2,3,1.0
6,3,1,1.0
7,3,2,0.0
8,3,3,1.0


Negative sampling 방법을 count-based 집계 스키마에 적용하면, 1이 아닌 0부터 count를 시작하고 0은 상호작용이 없는 것을 뜻하게 만들어 적용할 수 있다.